In [1]:
import evaluate
import json
import librosa
import pytesseract
import soundfile

import numpy as np
import tensorflow as tf

from datasets import load_dataset
from huggingface_hub import notebook_login
from pytesseract import Output
from transformers import pipeline
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import create_optimizer, AdamWeightDecay, TFAutoModelForSeq2SeqLM
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback
from transformers.pipelines.pt_utils import KeyDataset

/Users/sindhumadhavan/GitHub/transformers/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


KeyboardInterrupt: 

[Link to Hugging Face Dataset with Channel Messages](https://huggingface.co/datasets/pszemraj/midjourney-messages-cleaned)

```
data_files = { "train" : [
    'train-00000-of-00003.parquet'
    , 'train-00001-of-00003.parquet'
    , 'train-00002-of-00003.parquet'
]}
```

In [69]:
deduped_dataset = load_dataset("pszemraj/midjourney-messages-cleaned", data_dir="deduped", split="train", verification_mode="no_checks")
# NonMatchingSplitsSizesError fixed by setting verification code to none
# https://discuss.huggingface.co/t/nonmatchingsplitssizeserror/30033/4
# dataset = load_dataset("pszemraj/midjourney-messages-cleaned", split="train")


Generating train split:  75%|███████▌  | 14828769/19716685 [00:06<00:02, 2368551.97 examples/s]


In [71]:
deduped_dataset

Dataset({
    features: ['id', 'channel_id', 'text'],
    num_rows: 14828769
})

In [72]:
random.choice(deduped_dataset)

{'id': '1116300648032514139',
 'channel_id': '1008571023047798794',
 'text': 'elephant street art, graffiti'}

In [73]:
deduped_dataset.sort("channel_id")[:1]

{'id': ['1105917785852362823'],
 'channel_id': ['1008571023047798794'],
 'text': ['einen gutschein mit den Angaben von und an und Betrag']}

In [74]:
messages = deduped_dataset

In [75]:
messages = messages.train_test_split(test_size=0.2)

In [76]:
messages["train"][0]

{'id': '1088604617031561327',
 'channel_id': '989268300473192561',
 'text': 'soldado de la segunda guerra mundial de anime'}